In [ ]:



def get_bands_from_playlist_search(query="death metal 1990s", playlist_limit=5, track_limit_per_playlist=100):
    # Step 1: Search for playlists
    results = sp.search(q=query, type='playlist', limit=playlist_limit)
    playlists = results.get("playlists", {}).get("items", [])
    all_artists = set()

    for playlist in playlists:
        print(type(playlist))
        if not playlist:
            continue
        playlist_id = playlist["id"]
        print(f"📜 Playlist: {playlist['name']}")

        # Step 2: Fetch tracks from the playlist
        offset = 0
        while offset < track_limit_per_playlist:
            tracks = sp.playlist_tracks(playlist_id, limit=100, offset=offset)
            items = tracks.get("items", [])
            if not items:
                break
            for item in items:
                track = item.get("track")
                if not track:
                    continue
                for artist in track.get("artists", []):
                    all_artists.add(artist["name"])
            offset += 100

    # Step 3: Sort and return
    artist_list = sorted(all_artists)
    for artist in artist_list:
        print("-", artist)
    return artist_list
    

In [44]:
from collections import Counter

def get_frequent_artists_from_playlists(query, playlist_limit=5, track_limit=100, top_n=20):
    results = sp.search(q=query, type='playlist', limit=playlist_limit)
    playlists = [pl for pl in results['playlists']['items'] if pl]

    artist_counter = Counter()

    for playlist in playlists:
        playlist_id = playlist["id"]
        tracks = sp.playlist_tracks(playlist_id, limit=track_limit)["items"]
        for item in tracks:
            track = item.get("track")
            if track:
                for artist in track.get("artists", []):
                    artist_name = artist["name"]
                    artist_counter[artist_name] += 1

    return [artist for artist, _ in artist_counter.most_common(top_n)]
popular_bands = get_frequent_artists_from_playlists("Rock 1990s", top_n=20)
print("Popular bands in the genre:")
for band in popular_bands:
    print("-", band)

Popular bands in the genre:
- Oasis
- Radiohead
- Blur
- Ash
- Feeder
- Suede
- Pulp
- Supergrass
- The Verve
- Travis
- The Charlatans
- Manic Street Preachers
- Shed Seven
- Delirious?
- Primal Scream
- Kula Shaker
- The Stone Roses
- James
- The Lightning Seeds
- Ocean Colour Scene


In [5]:
from collections import Counter
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
from dotenv import load_dotenv
import os

load_dotenv()
CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
))

def fetch_all_tracks(playlist_id, total_limit=500):
    tracks = []
    offset = 0
    while offset < total_limit:
        response = sp.playlist_tracks(playlist_id, limit=100, offset=offset)
        items = response.get("items", [])
        if not items:
            break
        tracks.extend(items)
        offset += 100
    return tracks

def get_frequent_artists_from_playlists(query, playlist_limit=15, track_limit=250, top_n=20, genre_filter=None):
    results = sp.search(q=query, type='playlist', limit=playlist_limit)
    playlists = [pl for pl in results['playlists']['items'] if pl]

    artist_counter = Counter()
    artist_id_to_name = {}

    for playlist in playlists:
        playlist_id = playlist["id"]
        tracks = fetch_all_tracks(playlist_id, total_limit=track_limit)
        for item in tracks:
            track = item.get("track")
            if track:
                for artist in track.get("artists", []):
                    artist_id = artist["id"]
                    artist_name = artist["name"]
                    artist_counter[artist_id] += 1
                    artist_id_to_name[artist_id] = artist_name

    top_artists = artist_counter.most_common(top_n * 3)  # Overfetch to filter later
    enriched_artists = []

    for artist_id, count in top_artists:
        try:
            artist_obj = sp.artist(artist_id)
            name = artist_obj["name"]
            popularity = artist_obj.get("popularity", 0)
            genres = artist_obj.get("genres", [])
            if genre_filter and not any(genre_filter.lower() in g.lower() for g in genres):
                continue  # Skip artist not matching genre
            enriched_artists.append((name, popularity, count))
            if len(enriched_artists) == top_n:
                break
        except Exception as e:
            continue
    return enriched_artists


# Main loop
os.makedirs("artists", exist_ok=True)
genres = ["Rock", "Metal", "Bollywood", "Alternative Rock", "Nu Metal"]
for genre in genres:
    file_path = f"artists/{genre}.txt".replace(" ", "_")
    print(f"\n🎸 Popular bands in {genre}:")
    popular_bands = get_frequent_artists_from_playlists(genre, top_n=20)
    for band, popularity, count in popular_bands:
        print(f"- {band} (Popularity: {popularity}, Tracks: {count})")
    with open(file_path, "w", encoding="utf-8") as f:
        for band, popularity, _ in popular_bands:
            if popularity > 69: 
                f.write(band + "\n")


🎸 Popular bands in Rock:
- Sleepyheadz (Popularity: 72, Tracks: 41)
- Nursery Rhymes ABC (Popularity: 70, Tracks: 19)
- Red Hot Chili Peppers (Popularity: 85, Tracks: 12)
- The Rolling Stones (Popularity: 80, Tracks: 12)
- Guns N' Roses (Popularity: 82, Tracks: 12)
- Bill Conti (Popularity: 57, Tracks: 12)
- AC/DC (Popularity: 83, Tracks: 11)
- Queen (Popularity: 85, Tracks: 11)
- Linkin Park (Popularity: 90, Tracks: 9)
- Aerosmith (Popularity: 77, Tracks: 9)
- Bon Jovi (Popularity: 80, Tracks: 9)
- Hurricane on Saturn (Popularity: 42, Tracks: 9)
- Green Day (Popularity: 83, Tracks: 8)
- Fleetwood Mac (Popularity: 84, Tracks: 8)
- Nirvana (Popularity: 84, Tracks: 7)
- Bruce Springsteen (Popularity: 78, Tracks: 7)
- Survivor (Popularity: 70, Tracks: 7)
- The Beatles (Popularity: 85, Tracks: 7)
- Led Zeppelin (Popularity: 79, Tracks: 6)
- Van Halen (Popularity: 74, Tracks: 6)

🎸 Popular bands in Metal:
- Metallica (Popularity: 84, Tracks: 130)
- Iron Maiden (Popularity: 74, Tracks: 45)
